In [2]:
#https://github.com/openai/CLIP/issues/57

import os
import torch
import glob
from PIL import Image
import random
import clip
from tqdm.notebook import tqdm
import numpy as np
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm.notebook import tqdm
from torch.distributions.gamma import Gamma

EPOCH =10
BATCH_SIZE =16
# Parameters
a_u = 1
b_u = 0

a_minus = 10
b_minus = 0

a_plus = 5
b_plus = 0

iters = 2

device = "cuda:2" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

class cocodtrain(torch.utils.data.Dataset):
    def __init__(self, image_path='/home/jason/data/coco2014/images', text_path='/home/jason/data/coco2014/text', mode='train2014'):

        self.image_list = []
        self.image_list.extend(glob.glob(os.path.join(image_path, mode, '*.jpg')))
        self.image_list.sort()

        self.label_list = []
        self.label_list.extend(glob.glob(os.path.join(text_path, mode, '*.txt')))
        self.label_list.sort()

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        image = Image.open(self.image_list[index]).convert("RGB")
        image = image.resize((224,224), Image.BILINEAR)
        image = preprocess(image)
        #image = np.asarray(image)

        with open(self.label_list[index], "r") as f:
            data = f.readlines()
            label = random.choice(data)
            
        return image, label
trainset = cocodtrain('/home/jason/data/coco2014/images','/home/jason/data/coco2014/text','train2014')
trainloader = torch.utils.data.DataLoader(
                    trainset, 
                    batch_size=BATCH_SIZE,
                    shuffle=True, 
                    num_workers=16,
                    drop_last=True)

def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

#device = "cuda:3" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
#model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

#clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16
def sample_w(U, s_matrix):
    BS = s_matrix.shape[0]
    s_plus = s_matrix.masked_select(torch.eye(BS).bool())
    s_minus = s_matrix.masked_select(~torch.eye(BS).bool())
    w_plus_dist = Gamma(torch.tensor(1+a_plus).float().to(device),torch.tensor(U*s_plus + b_plus).float())
    w_minus_dist = Gamma(torch.tensor(a_minus).float().to(device),torch.tensor(U.unsqueeze(1)*s_minus + b_minus).float())
    w_plus = w_plus_dist.sample()
    w_minus = w_minus_dist.sample()
    tmp = torch.cat((w_plus[:-1].unsqueeze(1), w_minus.transpose(1, 0)), dim=1)
    w_matrix = torch.cat((tmp.view(-1), w_plus[-1].unsqueeze(0))).view(BS, BS)
    return w_matrix

    

def sample_u(w_matrix, sim_matrix):
    full_mat = w_matrix * sim_matrix
    rate_param = b_u + full_mat.sum(dim=1)
    u_dist = Gamma(torch.tensor(a_u).float().to(device), rate_param.float())
    return u_dist.sample()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

for epoch in range(EPOCH):
    print('epoch:', epoch)
    for batch in tqdm(trainloader):
        optimizer.zero_grad()
        list_image,list_txt = batch #list_images is list of image in numpy array(np.uint8), or list of PIL images
        # print(list_image.size()) #torch.Size([32, 3, 224, 224])
        print(len(list_txt))
      
        images = torch.tensor(np.stack(list_image)).to(device)
        texts = clip.tokenize(list_txt).to(device) #torch.Size([32, 77])
         # print(texts.size()) #torch.Size([32, 77])
        logits_per_image, logits_per_text = model(images, texts)
        # ground_truth = torch.arange(BATCH_SIZE,dtype=torch.long,device=device)

        # total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

        weights = torch.ones(BATCH_SIZE).to(device)
        for _ in range(iters):
            U = sample_u(weights, logits_per_image)
            weights = sample_w(U, logits_per_image)

        weighted_sim = weights * logits_per_image
        total_loss = loss_fn(weighted_sim, torch.arange(BATCH_SIZE).to(device))

        total_loss.backward()
        print('total loss:', total_loss)
      
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)
    
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': total_loss,
    }, f"model_checkpoint/model_10.pt") #just change to your preferred folder/filename      

epoch: 0


16


/tmp/ipykernel_915367/872034600.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_plus_dist = Gamma(torch.tensor(1+a_plus).float().to(device),torch.tensor(U*s_plus + b_plus).float())
/tmp/ipykernel_915367/872034600.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_minus_dist = Gamma(torch.tensor(a_minus).float().to(device),torch.tensor(U.unsqueeze(1)*s_minus + b_minus).float())


RuntimeError: Sizes of tensors must match except in dimension 0. Got 16 and 15 (The offending index is 0)

In [ ]:
# Two Implementation about Cross-Entropy


a = torch.rand(3, 3)
print(a)
loss = torch.nn.CrossEntropyLoss()
b = torch.arange(3)
print(b)
l1 = loss(a, b)
print(l1)

c = -torch.log(F.softmax(a, dim=1).masked_select(torch.eye(a.size(0), a.size(1), dtype=torch.bool)))

# print(-torch.log(F.softmax(a, dim=1)))
# print(c)

# diagonal_mask = torch.eye(matrix.size(0), matrix.size(1), dtype=torch.bool)
# d = matrix.masked_select(diagonal_mask)
print(c.mean())

In [23]:
import os
import torch
import glob
from PIL import Image
import random
import clip
from tqdm.notebook import tqdm
import numpy as np
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm.notebook import tqdm
from torch.distributions.gamma import Gamma
device = "cuda:2"
a_u = 1
b_u = 0

a_minus = 10
b_minus = 0

a_plus = 5
b_plus = 0

def sample_u(w_matrix, sim_matrix):
    full_mat = w_matrix * sim_matrix
    rate_param = b_u + full_mat.sum(dim=1)
    print("rate_param:", rate_param)
    u_dist = Gamma(torch.tensor(a_u).float().to(device), rate_param.float())
    return u_dist.sample()

def sample_w(U, s_matrix):
    BS = s_matrix.shape[0]
    s_plus = s_matrix.masked_select(torch.eye(BS).bool()).to(device)
    s_minus = s_matrix.masked_select(~torch.eye(BS).bool()).to(device)
    print("s_minus:", s_minus.size()) #torch.Size([12])

    w_plus_dist = Gamma(1+a_plus, (U*s_plus + b_plus))
    w_minus_dist = Gamma(a_minus, (U.unsqueeze(1)*s_minus + b_minus))
    print("U.unsqueeze:", (U.unsqueeze(1)*s_minus).size()) #torch.Size([4, 12])

    w_plus = w_plus_dist.sample()
    print("w_plus:", w_plus.size()) #torch.Size([4])
    w_minus = w_minus_dist.sample()
    print("w_minus:", w_minus.size()) #torch.Size([4, 12])
    print('w_plus[:-1]:', w_plus[:-1].size())

    tmp = torch.cat((w_plus[:-1].unsqueeze(1), w_minus.transpose(1, 0)), dim=1)

    
    w_matrix = torch.cat((tmp.view(-1), w_plus[-1].unsqueeze(0))).view(BS, 4)
    return w_matrix

BATCH_SIZE =4
weights = torch.ones(BATCH_SIZE, BATCH_SIZE).to(device)
similarity_score = torch.rand(BATCH_SIZE, BATCH_SIZE).to(device)
for _ in range(2):
    U = sample_u(weights, similarity_score)
    print("U:", U.size())
    weights = sample_w(U, similarity_score)
print(weights)

rate_param: tensor([2.7797, 1.5095, 2.2953, 1.5791], device='cuda:2')
U: torch.Size([4])
s_minus: torch.Size([12])
U.unsqueeze: torch.Size([4, 12])
w_plus: torch.Size([4])
w_minus: torch.Size([4, 12])
w_plus[:-1]: torch.Size([3])


RuntimeError: Sizes of tensors must match except in dimension 0. Got 12 and 3 (The offending index is 0)

In [51]:
import os
import torch
import glob
from PIL import Image
import random
import clip
from tqdm.notebook import tqdm
import numpy as np
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm.notebook import tqdm
from torch.distributions.gamma import Gamma
device = "cuda:2"
a_u = 1
b_u = 0

a_minus = 10
b_minus = 0

a_plus = 5
b_plus = 0

def sample_u(w_matrix, sim_matrix):
    full_mat = w_matrix * sim_matrix
    rate_param = b_u + full_mat.sum(dim=1)
    print("rate_param:", rate_param)
    u_dist = Gamma(torch.tensor(a_u).float().to(device), rate_param.float())
    return u_dist.sample()

def sample_w(U, s_matrix):
    BS = s_matrix.shape[0]
    s_plus = s_matrix.masked_select(torch.eye(BS).bool()).to(device)
    s_minus = s_matrix.masked_select(~torch.eye(BS).bool()).to(device)
    print("s_minus:", s_minus.size()) #torch.Size([12])
    print("U:", U)
    w_plus_dist = Gamma(1+a_plus, (U*s_plus + b_plus))
    U = U.repeat_interleave(int(len(s_minus)/BS))
    print("U:", U)



    w_minus_dist = Gamma(a_minus, (U*s_minus + b_minus))



    w_plus = w_plus_dist.sample()
    print("w_plus:", w_plus.size()) #torch.Size([4])
    w_minus = w_minus_dist.sample()
    print("w_minus:", w_minus.size()) #torch.Size([4, 12])
    print('w_plus[:-1]:', w_plus[:-1].size())

    result = torch.zeros(BS, BS).to(device)
    diagonal_matrix = torch.diag(w_plus)

    # Add the diagonal matrix to the zero matrix to create result
    result += diagonal_matrix
    print(result)

    # Fill b's elements sequentially into the remaining positions of result
    mask = ~torch.eye(BS, dtype=bool)  # Create a boolean mask that excludes the diagonal
    print(mask)
    result[mask] = w_minus

    # tmp = torch.cat((w_plus[:-1].unsqueeze(1), w_minus.transpose(1, 0)), dim=1)

    
    # w_matrix = torch.cat((tmp.view(-1), w_plus[-1].unsqueeze(0))).view(BS, 4)
    return result

BATCH_SIZE =4
weights = torch.ones(BATCH_SIZE, BATCH_SIZE).to(device)
similarity_score = torch.rand(BATCH_SIZE, BATCH_SIZE).to(device)
for _ in range(2):
    U = sample_u(weights, similarity_score)
    print("U:", U.size())
    weights = sample_w(U, similarity_score)
print(weights)

rate_param: tensor([2.5934, 2.1994, 1.6663, 1.8559], device='cuda:2')
U: torch.Size([4])
s_minus: torch.Size([12])
U: tensor([0.1638, 0.0477, 5.3180, 0.5599], device='cuda:2')
U: tensor([0.1638, 0.1638, 0.1638, 0.0477, 0.0477, 0.0477, 5.3180, 5.3180, 5.3180,
        0.5599, 0.5599, 0.5599], device='cuda:2')
w_plus: torch.Size([4])
w_minus: torch.Size([12])
w_plus[:-1]: torch.Size([3])
tensor([[ 20.1765,   0.0000,   0.0000,   0.0000],
        [  0.0000, 115.2418,   0.0000,   0.0000],
        [  0.0000,   0.0000,  26.2004,   0.0000],
        [  0.0000,   0.0000,   0.0000,  34.0093]], device='cuda:2')
tensor([[False,  True,  True,  True],
        [ True, False,  True,  True],
        [ True,  True, False,  True],
        [ True,  True,  True, False]])
rate_param: tensor([183.1842, 633.2631,   6.7449,  46.6484], device='cuda:2')
U: torch.Size([4])
s_minus: torch.Size([12])
U: tensor([0.0007, 0.0006, 0.1155, 0.0384], device='cuda:2')
U: tensor([0.0007, 0.0007, 0.0007, 0.0006, 0.0006, 0.0006

In [49]:
import torch

# Create example tensors a and b
a = torch.tensor([1, 2, 3, 4])
b = torch.tensor([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]).float()

# Create a 4x4 zero tensor result
result = torch.zeros(4, 4)
diagonal_matrix = torch.diag(a)

# Add the diagonal matrix to the zero matrix to create result
result += diagonal_matrix
print(result)

# Fill b's elements sequentially into the remaining positions of result
b_idx = 0
mask = ~torch.eye(4, dtype=bool)  # Create a boolean mask that excludes the diagonal
print(mask)
result[mask] = b
# result = result.int()  # Convert the result to integer type

# Output the result
print("a's elements on the diagonal:")
print(result)


tensor([[1., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 4.]])
tensor([[False,  True,  True,  True],
        [ True, False,  True,  True],
        [ True,  True, False,  True],
        [ True,  True,  True, False]])
a's elements on the diagonal:
tensor([[ 1.,  5.,  6.,  7.],
        [ 8.,  2.,  9., 10.],
        [11., 12.,  3., 13.],
        [14., 15., 16.,  4.]])


In [34]:
import torch

# 创建一个示例的浮点数张量
a = torch.tensor([1.0, 2.5, 3.7])

# 对整个张量重复3次
repeated_tensor = a.repeat(3)

# 输出结果
print("原始张量：", a)
print("重复后的张量：", repeated_tensor)


原始张量： tensor([1.0000, 2.5000, 3.7000])
重复后的张量： tensor([1.0000, 2.5000, 3.7000, 1.0000, 2.5000, 3.7000, 1.0000, 2.5000, 3.7000])
